In [1]:
import os
import struct
import numpy as np
import gc
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.losses import categorical_crossentropy

import matplotlib.pyplot as plt
%matplotlib inline

e:\ProgramData\Miniconda3\envs\tfgpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 1 . Prepare Dataset -- add random noise category

In [2]:
def shuffle(X_train,y_train):
    ind_list = [i for i in range(len(X_train))]
    np.random.shuffle(ind_list)
    X_train  = X_train[ind_list, :,:,:]
    y_train = y_train[ind_list,]
    
    return X_train,y_train

In [3]:
def gen_add_noise(X_train,y_train):
    noise_random = np.random.rand(50000,32,32,3)
    noise_labels = np.full((50000,1), 10, dtype=int)
    X_train = np.concatenate((X_train, noise_random), axis = 0 )
    y_train = np.concatenate((y_train, noise_labels), axis = 0 )
    
    return X_train,y_train

In [4]:
num_classes = 10

In [5]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data() 

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255
X_test  = X_test / 255

y_train = np_utils.to_categorical(y_train, num_classes)
y_test  = np_utils.to_categorical(y_test, num_classes)


In [6]:
# X_train, y_train = gen_add_noise(X_train,y_train)
# X_train, y_train = shuffle(X_train,y_train)

In [7]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck','Noise']

In [8]:
# fig = plt.figure(figsize=(10,5))
# for i in range(num_classes):
#     ax = fig.add_subplot(2, 6, 1 + i, xticks=[], yticks=[])
#     idx = np.where(y_train[:]==i)[0]
#     features_idx = X_train[idx,::]
#     img_num = np.random.randint(features_idx.shape[0])
#     im = features_idx[img_num,::]
#     ax.set_title(class_names[i])
#     plt.imshow(im)
# plt.show()

# 2 . Build Model -- keras model

In [9]:
def model_factory():
    K.clear_session()

    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     data_format='channels_last',
                     input_shape=(32, 32, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [10]:
# model = model_factory()
# model.summary()

# 3. Training

In [11]:
hs_history = []
hs_test_scores = []
for i in range(30):
    print("*********** Training for %d time **************" % (i+1))
    print()
    model = model_factory()
    
    history = model.fit(X_train, y_train, epochs=12, batch_size=32, validation_split = 0.1, verbose = 1)
    
    hs_history.append(history) 
    
    model.save("./models/CIFAR10_0420_random_kerasexample_%d.h5" % (i+1))

    test_score = model.evaluate(X_test, y_test, verbose=1)

    hs_test_scores.append(test_score)
    
    print()
    print("CIFAR10 test sets -> Loss: %f" % (test_score[0]))
    print("CIFAR10 test sets -> Accuracy: %f%%" % (test_score[1]*100))
    print()
    
    gc.collect()

*********** Training for 1 time **************

Train on 45000 samples, validate on 5000 samples
Epoch 1/12
45000/45000 [==============================] - 26s 577us/step - loss: 1.8672 - acc: 0.3178 - val_loss: 1.6108 - val_acc: 0.4214
Epoch 2/12
45000/45000 [==============================] - 23s 509us/step - loss: 1.5405 - acc: 0.4436 - val_loss: 1.4525 - val_acc: 0.4798
Epoch 3/12
45000/45000 [==============================] - 24s 530us/step - loss: 1.3931 - acc: 0.4999 - val_loss: 1.3018 - val_acc: 0.5286
Epoch 4/12
45000/45000 [==============================] - 24s 538us/step - loss: 1.2992 - acc: 0.5387 - val_loss: 1.2316 - val_acc: 0.5650
Epoch 5/12
45000/45000 [==============================] - 26s 571us/step - loss: 1.2228 - acc: 0.5673 - val_loss: 1.1498 - val_acc: 0.5912
Epoch 6/12
45000/45000 [==============================] - 24s 529us/step - loss: 1.1508 - acc: 0.5928 - val_loss: 1.0707 - val_acc: 0.6222
Epoch 7/12
45000/45000 [==============================] - 27s 600us/s

45000/45000 [==============================] - 24s 536us/step - loss: 1.3737 - acc: 0.5047 - val_loss: 1.2452 - val_acc: 0.5664
Epoch 4/12
45000/45000 [==============================] - 24s 540us/step - loss: 1.2807 - acc: 0.5434 - val_loss: 1.1502 - val_acc: 0.6042
Epoch 5/12
45000/45000 [==============================] - 24s 529us/step - loss: 1.2034 - acc: 0.5732 - val_loss: 1.1809 - val_acc: 0.5884
Epoch 6/12
45000/45000 [==============================] - 24s 532us/step - loss: 1.1346 - acc: 0.6000 - val_loss: 1.0358 - val_acc: 0.6366
Epoch 7/12
45000/45000 [==============================] - 24s 535us/step - loss: 1.0772 - acc: 0.6231 - val_loss: 1.0571 - val_acc: 0.6352
Epoch 8/12
45000/45000 [==============================] - 24s 533us/step - loss: 1.0292 - acc: 0.6405 - val_loss: 1.0126 - val_acc: 0.6592
Epoch 9/12
45000/45000 [==============================] - 24s 531us/step - loss: 0.9876 - acc: 0.6553 - val_loss: 0.9750 - val_acc: 0.6646
Epoch 10/12
45000/45000 [=============

45000/45000 [==============================] - 23s 520us/step - loss: 1.0990 - acc: 0.6134 - val_loss: 1.0441 - val_acc: 0.6378
Epoch 8/12
45000/45000 [==============================] - 23s 501us/step - loss: 1.0482 - acc: 0.6315 - val_loss: 0.9579 - val_acc: 0.6632
Epoch 9/12
45000/45000 [==============================] - 23s 510us/step - loss: 1.0100 - acc: 0.6463 - val_loss: 0.9514 - val_acc: 0.6690
Epoch 10/12
45000/45000 [==============================] - 23s 511us/step - loss: 0.9754 - acc: 0.6590 - val_loss: 0.9032 - val_acc: 0.6844
Epoch 11/12
45000/45000 [==============================] - 23s 512us/step - loss: 0.9416 - acc: 0.6699 - val_loss: 0.8521 - val_acc: 0.7048
Epoch 12/12
10000/10000 [==============================] - 2s 187us/step

CIFAR10 test sets -> Loss: 0.916363
CIFAR10 test sets -> Accuracy: 68.420000%

*********** Training for 10 time **************

Train on 45000 samples, validate on 5000 samples
Epoch 1/12
45000/45000 [==============================] - 24s 5

45000/45000 [==============================] - 24s 525us/step - loss: 0.9406 - acc: 0.6724 - val_loss: 0.9502 - val_acc: 0.6718
Epoch 12/12
10000/10000 [==============================] - 2s 189us/step

CIFAR10 test sets -> Loss: 0.957901
CIFAR10 test sets -> Accuracy: 66.360000%

*********** Training for 14 time **************

Train on 45000 samples, validate on 5000 samples
Epoch 1/12
45000/45000 [==============================] - 24s 531us/step - loss: 1.8324 - acc: 0.3262 - val_loss: 1.7002 - val_acc: 0.3836
Epoch 2/12
45000/45000 [==============================] - 24s 524us/step - loss: 1.5345 - acc: 0.4427 - val_loss: 1.4402 - val_acc: 0.4824
Epoch 3/12
45000/45000 [==============================] - 23s 511us/step - loss: 1.3865 - acc: 0.5008 - val_loss: 1.2909 - val_acc: 0.5356
Epoch 4/12
45000/45000 [==============================] - 23s 514us/step - loss: 1.2839 - acc: 0.5411 - val_loss: 1.2295 - val_acc: 0.5662
Epoch 5/12
45000/45000 [==============================] - 23s 517

45000/45000 [==============================] - 23s 519us/step - loss: 1.8548 - acc: 0.3229 - val_loss: 1.6594 - val_acc: 0.4080
Epoch 2/12
45000/45000 [==============================] - 25s 555us/step - loss: 1.5510 - acc: 0.4362 - val_loss: 1.4181 - val_acc: 0.4908
Epoch 3/12
45000/45000 [==============================] - 23s 522us/step - loss: 1.4079 - acc: 0.4910 - val_loss: 1.3893 - val_acc: 0.4982
Epoch 4/12
45000/45000 [==============================] - 25s 552us/step - loss: 1.3078 - acc: 0.5304 - val_loss: 1.2782 - val_acc: 0.5486
Epoch 5/12
45000/45000 [==============================] - 25s 550us/step - loss: 1.2336 - acc: 0.5620 - val_loss: 1.1574 - val_acc: 0.5866
Epoch 6/12
45000/45000 [==============================] - 24s 527us/step - loss: 1.1631 - acc: 0.5867 - val_loss: 1.0499 - val_acc: 0.6316
Epoch 7/12
45000/45000 [==============================] - 25s 545us/step - loss: 1.1043 - acc: 0.6117 - val_loss: 1.0742 - val_acc: 0.6238
Epoch 8/12
45000/45000 [==============

45000/45000 [==============================] - 27s 603us/step - loss: 1.1917 - acc: 0.5768 - val_loss: 1.2325 - val_acc: 0.5662
Epoch 6/12
45000/45000 [==============================] - 27s 597us/step - loss: 1.1273 - acc: 0.6028 - val_loss: 1.0508 - val_acc: 0.6302
Epoch 7/12
45000/45000 [==============================] - 28s 619us/step - loss: 1.0726 - acc: 0.6211 - val_loss: 1.0230 - val_acc: 0.6368
Epoch 8/12
45000/45000 [==============================] - 25s 564us/step - loss: 1.0277 - acc: 0.6390 - val_loss: 1.2737 - val_acc: 0.5636
Epoch 9/12
45000/45000 [==============================] - 24s 539us/step - loss: 0.9865 - acc: 0.6543 - val_loss: 0.8975 - val_acc: 0.6888
Epoch 10/12
45000/45000 [==============================] - 24s 531us/step - loss: 0.9517 - acc: 0.6676 - val_loss: 0.8785 - val_acc: 0.6914
Epoch 11/12
45000/45000 [==============================] - 24s 534us/step - loss: 0.9212 - acc: 0.6769 - val_loss: 0.9251 - val_acc: 0.6802
Epoch 12/12
10000/10000 [===========

45000/45000 [==============================] - 24s 541us/step - loss: 1.0388 - acc: 0.6357 - val_loss: 0.9439 - val_acc: 0.6722
Epoch 10/12
45000/45000 [==============================] - 24s 541us/step - loss: 0.9945 - acc: 0.6519 - val_loss: 1.0266 - val_acc: 0.6408
Epoch 11/12
45000/45000 [==============================] - 23s 502us/step - loss: 0.9597 - acc: 0.6639 - val_loss: 0.9442 - val_acc: 0.6700
Epoch 12/12
10000/10000 [==============================] - 2s 246us/step

CIFAR10 test sets -> Loss: 0.944260
CIFAR10 test sets -> Accuracy: 66.810000%

*********** Training for 27 time **************

Train on 45000 samples, validate on 5000 samples
Epoch 1/12
45000/45000 [==============================] - 25s 561us/step - loss: 1.8451 - acc: 0.3246 - val_loss: 1.5730 - val_acc: 0.4446
Epoch 2/12
45000/45000 [==============================] - 26s 584us/step - loss: 1.5283 - acc: 0.4431 - val_loss: 1.3837 - val_acc: 0.5064
Epoch 3/12
45000/45000 [==============================] - 27s 6

10000/10000 [==============================] - 2s 191us/step

CIFAR10 test sets -> Loss: 1.017160
CIFAR10 test sets -> Accuracy: 64.960000%



In [12]:
hs_test_scores

[[0.9391200876235962, 0.6741],
 [0.8478991918563843, 0.7001],
 [0.8895312670707702, 0.6906],
 [0.8611604751586914, 0.6994],
 [0.8633020091056823, 0.7037],
 [0.8614060991287231, 0.6975],
 [0.8658420544624329, 0.7013],
 [0.9913038634300232, 0.6688],
 [0.9163629177093506, 0.6842],
 [0.9104057406425476, 0.6815],
 [0.8369299451828003, 0.7107],
 [0.948795693397522, 0.6679],
 [0.9579006961822509, 0.6636],
 [0.8943304543495179, 0.6867],
 [0.8819027346611022, 0.6933],
 [0.9187321332931518, 0.6795],
 [0.878794640827179, 0.694],
 [0.8931140807151794, 0.6901],
 [0.8656606731414795, 0.6987],
 [0.8695392421722412, 0.6967],
 [0.9751727300643921, 0.6664],
 [0.8484252559661866, 0.7047],
 [0.8867856386184693, 0.6942],
 [0.8765434796333313, 0.6923],
 [0.8729298974990845, 0.6945],
 [0.9442604482650757, 0.6681],
 [0.8688526017189026, 0.7001],
 [0.8623462996482849, 0.7014],
 [0.8947820237159729, 0.6893],
 [1.0171595403671265, 0.6496]]